- PDF 파일을 openAI 의 API 를 사용하여 RAG 검색 

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
def langsmith(project_name=None, set_enable=True):

    if set_enable:
        langchain_key = os.environ.get("LANGCHAIN_API_KEY", "")
        langsmith_key = os.environ.get("LANGSMITH_API_KEY", "")

        # 더 긴 API 키 선택
        if len(langchain_key.strip()) >= len(langsmith_key.strip()):
            result = langchain_key
        else:
            result = langsmith_key

        if result.strip() == "":
            print(
                "LangChain/LangSmith API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954"
            )
            return

        os.environ["LANGSMITH_ENDPOINT"] = (
            "https://api.smith.langchain.com"  # LangSmith API 엔드포인트
        )
        os.environ["LANGSMITH_TRACING"] = "true"  # true: 활성화
        os.environ["LANGSMITH_PROJECT"] = project_name  # 프로젝트명
        print(f"LangSmith 추적을 시작합니다.\n[프로젝트명]\n{project_name}")
    else:
        os.environ["LANGSMITH_TRACING"] = "false"  # false: 비활성화
        print("LangSmith 추적을 하지 않습니다.")


def env_variable(key, value):
    os.environ[key] = value

In [3]:
# 프로젝트 이름 입력
langsmith("ChatGPT_RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
ChatGPT_RAG


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/AI기반_인파분석플랫폼구축_제안서.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

문서의 페이지수: 113
분할된 청크의수: 102


In [6]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [7]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [15]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [16]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("포항에서 열리는 축제 이름은?")

[Document(id='75f5b2fd-3ef2-4285-9a59-b65386609364', metadata={'producer': '알PDF', 'creator': '알PDF', 'creationdate': '2024-06-09T21:45:38+09:00', 'source': 'data/AI기반_인파분석플랫폼구축_제안서.pdf', 'file_path': 'data/AI기반_인파분석플랫폼구축_제안서.pdf', 'total_pages': 113, 'format': 'PDF 1.7', 'title': '합치기_2', 'author': '유선철', 'subject': '', 'keywords': '', 'moddate': '2024-06-09T21:57:34+09:00', 'trapped': '', 'page': 51}, page_content='SFR-002\nSec \n1-2\n위치: 경북포항시북구상대로\n59-1\n상태: 정상'),
 Document(id='0e524263-32c7-441a-877f-28ad7f27fe93', metadata={'producer': '알PDF', 'creator': '알PDF', 'creationdate': '2024-06-09T21:45:38+09:00', 'source': 'data/AI기반_인파분석플랫폼구축_제안서.pdf', 'file_path': 'data/AI기반_인파분석플랫폼구축_제안서.pdf', 'total_pages': 113, 'format': 'PDF 1.7', 'title': '합치기_2', 'author': '유선철', 'subject': '', 'keywords': '', 'moddate': '2024-06-09T21:57:34+09:00', 'trapped': '', 'page': 0}, page_content='접수\n번호\n「AI 기반 인파 분석 플랫폼 구축」\n제  안  서'),
 Document(id='2dddb607-3ca8-424f-adcb-68b918a39d86', metadata={'pr

In [10]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [11]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [12]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "포항에서 열리는 축제 이름은?"
response = chain.invoke(question)
print(response)

청년문화페스티벌


In [14]:
question = "청년 문화 페스티벌의 2024년 방문자수가 몇명이야?"
response = chain.invoke(question)
print(response)

모르겠습니다.


In [19]:
question = "문서에 피플카운팅에 관한 내용이 들어있어?"
response = chain.invoke(question)
print(response)

그 문서에는 피플카운팅에 관한 내용이 포함되어 있습니다. "구역별 유동인구 수집 및 분석"이라는 항목이 언급되어 있습니다.


In [20]:
question = "몇 페이지야?"
response = chain.invoke(question)
print(response)

모르겠습니다.


In [21]:
question = "피플 카운팅에 관한 내용은 몇 페이지에 있어?"
response = chain.invoke(question)
print(response)

그에 대한 정보는 제공된 문서에서 찾을 수 없습니다.
